In [ ]:
# ╔══════════ 0. Drive + зависимости (ничего не фиксируем) ═════════╗
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
# чтобы избежать фрагментации и расширять сегменты под PyTorch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
!pip -q install transformers peft bitsandbytes accelerate datasets sentencepiece tqdm pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 20.9 MB/s eta 0:00:00


In [ ]:
# ╔══════════ 1. Пути и базовые параметры ═══════════════════════════╗
DATA_FILE   = "/content/drive/MyDrive/finetuning_text_pairs_3_clean_v2.csv"
BASE_ID     = "Vikhrmodels/Vikhr-Qwen-2.5-1.5B-Instruct"
SAVE_DIR    = "/content/drive/MyDrive/vikhr_qwen_finetuned_fix"
VAL_SPLIT   = 0.10
SEED        = 42
MAXLEN      = 1024          # обучаемся на том же контексте
BATCH       = 1             # per-device; T4/L4 хватает
GA_STEPS    = 16            # → effective BS = 32

In [ ]:
# ╔══════════ 2. Чтение и чистка датасета ═══════════════════════════╗
import pandas as pd, numpy as np
df = pd.read_csv(DATA_FILE).dropna(subset=["finetuning_text_pairs_3","Unnamed: 1"])
df = df.rename(columns={"finetuning_text_pairs_3":"src","Unnamed: 1":"tgt"})
df["src"] = df["src"].astype(str)
df["tgt"] = df["tgt"].astype(str)

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=VAL_SPLIT, random_state=SEED)

from datasets import Dataset
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df .reset_index(drop=True))

In [ ]:
# ╔══════════ 3. Токенизатор и правильная препроцесс-функция ═════════╗
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained(BASE_ID, trust_remote_code=True)
tok.pad_token = tok.eos_token        # GPT-семейство не имеет PAD

PROMPT = "Упрости текст, сохраняя смысл:\n\n{src}\n\nУпрощённая версия:"

def preprocess(batch):
    inputs, labels = [], []
    for s, t in zip(batch["src"], batch["tgt"]):
        p_ids = tok(PROMPT.format(src=s),
                    truncation=True, max_length=MAXLEN,
                    add_special_tokens=False).input_ids
        t_ids = tok(t, truncation=True, max_length=MAXLEN-1,
                    add_special_tokens=False).input_ids + [tok.eos_token_id]
        # объединяем и ограничиваем
        ids   = (p_ids + t_ids)[:MAXLEN]
        labs  = ([-100]*len(p_ids) + t_ids)[:MAXLEN]  # -100 на промпт
        inputs.append(ids); labels.append(labs)
    return {"input_ids":inputs, "labels":labels}

train_ds = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
val_ds   = val_ds.map(preprocess,   batched=True, remove_columns=val_ds.column_names)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

Map:   0%|          | 0/2576 [00:00<?, ? examples/s]

Map:   0%|          | 0/287 [00:00<?, ? examples/s]

In [ ]:
# ╔══════════ 4. Модель 4-bit + LoRA ════════════════════════════════╗
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

base = AutoModelForCausalLM.from_pretrained(
    BASE_ID,
    quantization_config=bnb_cfg,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
base.gradient_checkpointing_enable()

lora_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj"],
    task_type="CAUSAL_LM", bias="none"
)
model = get_peft_model(base, lora_cfg)
model.print_trainable_parameters()

model.enable_input_require_grads()

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

trainable params: 4,530,176 || all params: 1,547,828,224 || trainable%: 0.2927


In [ ]:
# ╔══════════ 5. TrainingArguments + Trainer ════════════════════════╗
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
args = TrainingArguments(
    output_dir="/content/qwen_lora_out",
    num_train_epochs=5,
    per_device_train_batch_size=BATCH,
    gradient_accumulation_steps=GA_STEPS,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=50,
    fp16=True,
    bf16=False,
    report_to="none",
)

collator = DataCollatorForSeq2Seq(tok, model=model,
                                  pad_to_multiple_of=8,
                                  label_pad_token_id=-100)
trainer = Trainer(model=model, args=args,
                  train_dataset=train_ds, eval_dataset=val_ds,
                  data_collator=collator)

trainer.train()

# ╔══════════ 6. Сохраняем LoRA-адаптер ══════════════════════════════╗
model.save_pretrained(SAVE_DIR)
tok.save_pretrained(SAVE_DIR)
print("✅ LoRA сохранена в:", SAVE_DIR)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,0.555900,0.543870
2,0.497300,0.530680
3,0.422500,0.539991
4,0.356100,0.564910
5,0.332100,0.580867


✅ LoRA сохранена в: /content/drive/MyDrive/vikhr_qwen_finetuned_fix


In [ ]:
# Необходимые библиотеки
!pip -q install transformers peft bitsandbytes accelerate pandas openpyxl tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import torch, pandas as pd, numpy as np, tqdm, time
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# Пути к файлам и моделям
BASE_ID       = "Vikhrmodels/Vikhr-Qwen-2.5-1.5B-Instruct"
ADAPTER_DIR   = "/content/drive/MyDrive/vikhr_qwen_finetuned_fix"
SRC_XLSX      = "/content/drive/MyDrive/unique_texts_200_formatted_golden_standard.xlsx"
DST_XLSX      = SRC_XLSX  # Записываем в тот же файл

In [ ]:
# Загружаем токенизатор
tok = AutoTokenizer.from_pretrained(BASE_ID, trust_remote_code=True)
tok.pad_token = tok.eos_token

# Загружаем базовую модель с 4-битным квантованием
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

try:
    # Пробуем загрузить с 4-бит квантованием
    base = AutoModelForCausalLM.from_pretrained(
        BASE_ID,
        quantization_config=bnb_cfg,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )
except Exception as e:
    print("⚠️ Не удалось загрузить с 4-бит квантованием:", e)
    # Fallback на fp16 без квантования
    base = AutoModelForCausalLM.from_pretrained(
        BASE_ID,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


⚠️ Не удалось загрузить с 4-бит квантованием: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
# Загружаем дообученную модель с адаптером LoRA
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()
device = next(model.parameters()).device
print("✅ Дообученная модель Qwen готова на", device)

# Промпт для генерации упрощенных текстов
PROMPT = "Упрости текст, сохраняя смысл:\n\n{src}\n\nУпрощённая версия:"

✅ Дообученная модель Qwen готова на cpu


In [ ]:
@torch.inference_mode()
def simplify(text, temp=0.3, top_p=0.9, max_new=512):
    """Генерирует упрощенную версию текста"""
    prompt = PROMPT.format(src=text)
    inputs = tok(prompt, return_tensors="pt").to(device)

    # Генерация с контролируемыми параметрами
    gen = model.generate(
        **inputs,
        do_sample=True,
        temperature=temp,
        top_p=top_p,
        max_new_tokens=max_new,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.eos_token_id,
        repetition_penalty=1.1,  # Небольшой штраф за повторения
    )

    # Извлекаем только сгенерированный текст после маркера
    output = tok.decode(gen[0], skip_special_tokens=True)
    if "Упрощённая версия:" in output:
        return output.split("Упрощённая версия:")[-1].strip()
    else:
        # Если маркер не найден, берем все что после входного текста
        return output[len(prompt) - len(text):].strip()

In [ ]:
# Читаем исходный файл
df = pd.read_excel(SRC_XLSX)
print(f"Загружено {len(df)} текстов из файла")

# Создаём колонку для результатов, если её ещё нет
col_name = "qwen_finetuned"
if col_name not in df.columns:
    df[col_name] = np.nan

# Преобразуем колонку в строковый тип для избежания проблем с типами
df[col_name] = df[col_name].astype('object')

# Генерируем упрощенные тексты с автосохранением
for i in tqdm.tqdm(range(len(df)), desc="Генерация упрощенных текстов"):
    # Генерируем только для пустых ячеек
    if pd.isna(df.at[i, col_name]):
        try:
            source_text = str(df.iloc[i, 0])
            print(f"\nОбрабатываю текст {i+1}/{len(df)}")

            # Пауза между обработками для стабильности
            if i > 0:
                time.sleep(1.0)

            # Генерируем упрощенный текст
            simplified = simplify(source_text)
            df.at[i, col_name] = simplified

            # Выводим для контроля результат
            print(f"Исходный текст: {source_text[:100]}...")
            print(f"Упрощенный:     {simplified[:100]}...")

        except Exception as err:
            print(f"❌ Ошибка при обработке строки {i}: {err}")
            df.at[i, col_name] = f"[ERROR: {err}]"

        # Сохраняем результат после каждого обработанного текста
df.to_excel(DST_XLSX, index=False)

print("🎉 Генерация завершена! Результаты сохранены в", DST_XLSX)

# Сохраняем все результаты в новый файл
NEW_RESULT_FILE = "/content/drive/MyDrive/unique_texts_200_qwen_results_final.xlsx"
df.to_excel(NEW_RESULT_FILE, index=False)
print(f"✅ Все результаты сохранены в {NEW_RESULT_FILE}")

# Проверяем, что файл содержит данные
check_df = pd.read_excel(NEW_RESULT_FILE)
print(f"В файле {len(check_df)} строк")
print(f"Заполнено результатами: {check_df[col_name].notna().sum()} строк из {len(check_df)}")

# Предлагаем скачать файл
from google.colab import files
files.download(NEW_RESULT_FILE)